In [1]:
!pip install tqdm

### Source code to generate most frequent used road and longest avg traversed time road

In [2]:
import geopandas as gp
import osmnx as ox
import numpy as np
import time
import csv
import os

import matplotlib.pyplot as plt

from tqdm import tqdm
from shapely.geometry import Polygon
from math import radians, cos, sin, asin, sqrt

In [3]:
edge_dataframe = gp.GeoDataFrame.from_file('porto_whole1/edges.shp')
#print('Total edges in the graph:' , len(edge_dataframe))
#edge_dataframe.head()

nodes_dataframe = gp.GeoDataFrame.from_file('porto_whole1/nodes.shp')
#print('Total nodes in the graph: ', len(nodes_dataframe))
#nodes_dataframe.head()

In [4]:
def get_edge_frequency():
    csv_data = open('porto_whole1/postprocessed_output.csv', 'r')
    dict_reader = csv.DictReader(csv_data)
    
    edge_frequnce = {}
    
    for c, item in enumerate(dict_reader):
        if item['match_success'] == 'FALSE':
            #print("dropping: ", c)
            continue
        #print(item['fmm-mapped edge indice sequence(map edge id)(equivalent to cpath from FMM)'])
        #print(c)
        edge_index = eval(item['fmm-mapped edge indice sequence(map edge id)(equivalent to cpath from FMM)'])
        for i in edge_index:
            if i in edge_frequnce.keys():
                edge_frequnce[i] = edge_frequnce[i] + 1
            else:
                edge_frequnce[i] = 1
        
    csv_data.close()
    return edge_frequnce

In [5]:
edge_freq = get_edge_frequency()
print(edge_freq)

{8752: 53, 941: 26, 1052: 25, 336: 6, 648: 6, 4840: 4, 6928: 3, 6920: 3, 6919: 3, 8594: 6, 446: 6, 6913: 6, 3083: 10, 451: 4, 4824: 2, 1177: 2, 1232: 2, 1209: 2, 1187: 2, 1206: 1, 1208: 1, 5058: 1, 1190: 1, 1194: 2, 7820: 4, 52: 33, 54: 39, 57: 24, 5134: 39, 7044: 21, 10366: 27, 740: 12, 3343: 12, 3340: 12, 5690: 11, 3342: 11, 584: 11, 3136: 11, 5282: 9, 5285: 9, 880: 10, 569: 10, 440: 22, 3171: 21, 441: 1, 3269: 1, 483: 10, 1061: 15, 481: 13, 238: 9, 3416: 9, 8462: 9, 3415: 13, 235: 7, 3399: 7, 5135: 7, 1156: 7, 2374: 7, 1051: 5, 709: 29, 711: 28, 7011: 27, 5679: 1, 5681: 6, 8235: 4, 7804: 2, 809: 1, 808: 5, 835: 46, 7712: 27, 8475: 25, 7710: 25, 755: 23, 3351: 23, 940: 63, 31: 22, 92: 23, 7797: 25, 90: 32, 3099: 28, 3095: 27, 3100: 35, 3109: 35, 3105: 27, 3097: 23, 10673: 28, 7051: 24, 4511: 22, 1042: 22, 875: 30, 886: 30, 871: 5, 878: 8, 8460: 10, 34: 9, 35: 53, 5645: 46, 5641: 45, 7813: 38, 7810: 41, 32: 45, 6745: 79, 1055: 75, 29: 56, 11059: 7, 4513: 2, 729: 52, 766: 47, 1678: 46,

In [6]:
class Edge(object):
    def __init__(self, idx, u, v, geometry):
        self.idx = idx
        self.u = u
        self.v = v
        self.geometry = geometry

In [7]:
def get_edge_from_idx(index):
    edge_data = edge_dataframe.iloc[index]
    u_pos = np.flatnonzero(nodes_dataframe['osmid']==edge_data.u)
    node_u_data = nodes_dataframe.iloc[u_pos]
    v_pos = np.flatnonzero(nodes_dataframe['osmid']==edge_data.v)
    node_v_data = nodes_dataframe.iloc[v_pos]
    
    u = (node_u_data.x.iloc[0], node_u_data.y.iloc[0])
    v = (node_v_data.x.iloc[0], node_v_data.y.iloc[0])
    geo = edge_data.geometry
    
    
    edge = Edge(index, u, v, geo)
    
    return edge

In [8]:
def dict_add_n(dict_to_add, key, n):
    if key in dict_to_add.keys():
        dict_to_add[key] += n
    else:
        dict_to_add[key] = n

In [9]:
def haversine_dist(coord1, coord2):
    lon1, lat1 = coord1[0], coord1[1]
    lon2, lat2 = coord2[0], coord2[1]
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [10]:
def get_edge_timeslice():
    csv_data = open('porto_whole1/postprocessed_output.csv', 'r')
    dict_reader = csv.DictReader(csv_data)
    
    edge_timeslice_list = []
    edge_timeslice_dict = {}
    
    processing_count = 1
    with tqdm(total=662) as _tqdm:
        for item in dict_reader:
            _tqdm.set_description('processing: {}/{} trajectory'.format(processing_count, 662))
            trajectory_edge_timeslice = {}
            if item['match_success'] == 'FALSE':
                continue
            raw_gps_trajectories = eval(item['raw_gps_trajectories'])
            edge_seq = eval(item['fmm-mapped edge indice sequence(map edge id)(equivalent to cpath from FMM)'])

            g_pointer = 0
            e_pointer = 0


            while g_pointer < len(raw_gps_trajectories):
                edge = get_edge_from_idx(edge_seq[e_pointer])
                gps_coord = raw_gps_trajectories[g_pointer]

                if e_pointer == len(edge_seq) - 1:
                    dict_add_n(trajectory_edge_timeslice, edge.idx, 1)
                    g_pointer += 1
                    continue

                if haversine_dist(gps_coord, edge.u) > haversine_dist(edge.u, edge.v):
                    e_pointer += 1
                    edge = get_edge_from_idx(edge_seq[e_pointer])

                dict_add_n(trajectory_edge_timeslice, edge.idx, 1)
                g_pointer += 1
                
            _tqdm.update(1)
            processing_count += 1
            edge_timeslice_list.append(trajectory_edge_timeslice)
        
    edge_timeslice_dict = compute_edge_timeslice(edge_timeslice_list)
    return edge_timeslice_dict

In [11]:
def compute_edge_timeslice(edge_timeslice_list):
    edge_timeslice_dict = {}
    for i in range(0, len(edge_dataframe)):
        count = 0
        for item in edge_timeslice_list:
            if i in item.keys():
                dict_add_n(edge_timeslice_dict, i, item[i])
                count += 1
        if i in edge_timeslice_dict.keys():
            edge_timeslice_dict[i] = edge_timeslice_dict[i] / count
    
    return edge_timeslice_dict   

In [12]:
edge_timeslice = get_edge_timeslice()

processing: 662/662 trajectory: 100%|███████████████████████████████████████████████▉| 661/662 [01:23<00:00,  7.87it/s]


In [13]:
def write_edge_time_freq(target_file_path, edge_timeslice, edge_frequency):
    target_csv = open(target_file_path, 'w', newline='')
    dict_writer = csv.writer(target_csv)
    
    file_header = ['edge index', 'traverse frequency', 'avg traverse time(s)', 'geometry', 'points sequence']
    dict_writer.writerow(file_header)
    
    for k in edge_frequency.keys():
        row = []
        row.append(k)
        row.append(edge_frequency[k])
        if k in edge_timeslice.keys():
            row.append(edge_timeslice[k] * 15)
        else:
            row.append('')
        geo = edge_dataframe.iloc[k].geometry
        row.append(geo)
        points = []
        for (x, y) in zip(geo.xy[0], geo.xy[1]):
            points.append([x, y])
        row.append(points)
        
        dict_writer.writerow(row)
    
    target_csv.close() 

In [14]:
write_edge_time_freq('data/edge_time_freq.csv', edge_timeslice, edge_freq)

### Using the code: Return First 5 most commonly used road segment

In [15]:
top5_keys = sorted(edge_freq, key=edge_freq.get, reverse=True)[:5]

for i, key in enumerate(top5_keys):
    print(i, " Road ID:", key)
    print("use count", edge_freq[key])
    print()

0  Road ID: 2134
use count 80

1  Road ID: 2760
use count 80

2  Road ID: 6745
use count 79

3  Road ID: 2802
use count 79

4  Road ID: 1055
use count 75



### Using the code: Return First 5 longest avg traverse road segment

In [16]:
top5_keys = sorted(edge_timeslice, key=edge_timeslice.get, reverse=True)[:5]

for i, key in enumerate(top5_keys):
    print(i, " Road ID:", key)
    print("avg traverse time", edge_timeslice[key] * 15) #one pt count is approx 15s 
    print()

0  Road ID: 928
avg traverse time 1890.0

1  Road ID: 6982
avg traverse time 637.5

2  Road ID: 3460
avg traverse time 480.0

3  Road ID: 9881
avg traverse time 470.0

4  Road ID: 7093
avg traverse time 420.0

